<a href="https://colab.research.google.com/github/Aadhisilambarasan/Academic-Query-Classification-using-Naive-Bayes/blob/main/Academic_Query_Classification_using_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'academic-query-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F67156%2F7555629%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240429%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240429T014536Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2cb87e9b7bf270a85158e9856751e523d0947073a3dd0e59725d4ffcffa04b07a6720b80b1dd6a3c2b72f47dcfadff607d9972a1c6111a22a810388f272f9868bb79233791101dcb05f6ed5abc764cfe9ebd0fdf75eb090c01cc183fff7a1557414e0acbc74700676370de97ea5cb9baf5f1d8643e5f81ecc708f872bcef26f17e044f2e986d68bc7816b24659bd5cf9e87418791d3f46f99051d99599c43c30623ad3d5f65d2d187391b3531310bc1309faace4cecc5483d2157396b56eb4ff0e3060ee7970c7e783c5aad0b2cf73ff8642cb50baa836dd5a84530d80ab5786fb5cdf15b990e1f6dc05a4050adc3c4ca475d4b4d4091d28c4366a31cc26f716'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Required Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Load and Explore Data

In [ ]:
# Load train and test data
train_data = pd.read_csv('/kaggle/input/academic-query-classification/train.csv')
test_data = pd.read_csv('/kaggle/input/academic-query-classification/test.csv')

In [ ]:
# Display the first few rows of train data
print("Train Data:")
print(train_data.head())

# Display the first few rows of test data
print("\nTest Data:")
print(test_data.head())

In [ ]:
# Check for missing values
print("Train Data Missing Values:")
print(train_data.isnull().sum())

print("\nTest Data Missing Values:")
print(test_data.isnull().sum())

In [ ]:
# Explore category distribution before balancing
plt.figure(figsize=(10,5))
category_counts=train_data['category'].value_counts()
plt.bar(category_counts.index, category_counts.values)
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Category Distribution Before Balancing')
plt.show()


# Preprocessing Data


### Balancing the data

In [ ]:
# balance data by resampling each class to have the same number of samples
# Balancing the data and silencing the warning
grouped_train_data = train_data.groupby('category', group_keys=False)
balanced_train_data = grouped_train_data.apply(lambda x: x.sample(train_data['category'].value_counts().min()).reset_index(drop=True))
balanced_train_data = balanced_train_data.reset_index(drop=True)

In [ ]:
# Explore category distribution after balancing
plt.figure(figsize=(10,5))
category_bal=balanced_train_data['category'].value_counts()
plt.bar(category_bal.index, category_bal.values)
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Category Distribution After Balancing')
plt.show()

In [ ]:
# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform train data
X_train = tfidf_vectorizer.fit_transform(balanced_train_data['query_text'])
y_train = balanced_train_data['category']

# Transform test data
X_test = tfidf_vectorizer.transform(test_data['query_text'])

# Training Naive Bayes Classifier

In [ ]:
# Initialize Naive Bayes classifier
nb_classifier = MultinomialNB()

# Train the classifier
nb_classifier.fit(X_train, y_train)


# Evaluating Classifier

In [ ]:
# Predict on train data for evaluation
train_predictions = nb_classifier.predict(X_train)

# Calculate accuracy
train_accuracy = accuracy_score(y_train, train_predictions)
print("Train Accuracy:", train_accuracy)

# Generate classification report
print("Classification Report:")
print(classification_report(y_train, train_predictions))

# Making Predictions

In [ ]:
# predictions on test data
test_predictions = nb_classifier.predict(X_test)


### Saving Predictions to CSV

In [ ]:
# Create a DataFrame for predictions
output = pd.DataFrame({'unique ID': test_data['unique ID'], 'category': test_predictions})

# Save predictions to CSV
output[['unique ID', 'category']].to_csv('/kaggle/working/submission.csv', index=False)